In [1]:
import submission
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import math
import matplotlib.pyplot as plt

In [2]:
## Parameters settings
past_cases_interval = 10
past_weather_interval = 10


## Read training data
train_file = './data/COVID_train_data.csv'
train_df = pd.read_csv(train_file)

## Read Training labels
train_label_file = './data/COVID_train_labels.csv'
train_labels_df = pd.read_csv(train_label_file)


## Read testing Features
test_fea_file = './data/test_features.csv'
test_features = pd.read_csv(test_fea_file)

test_label_file ='./data/COVID_test_labels.csv'
test_labels_df = pd.read_csv(test_label_file)
ground_truth = test_labels_df['dailly_cases'].to_list()



# # kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# # Degree of the polynomial kernel function for poly in kernl
# # gamma: scale or auto
# # C: Regularization parameter. The strength of the regularization is inversely proportional to C
# # coef0 useful in poly and sigmoid


# ## Set hyper-parameters for the SVM Model
# svm_model = SVR()
# svm_model.set_params(**{'kernel': 'poly', 'degree': 1, 'C': 5000,
#                         'gamma': 'scale', 'coef0': 0.0, 'tol': 0.001, 'epsilon': 10})


In [3]:
# get training labels 
# size 162x2
feature_matrix_y_for_max_interval = train_labels_df[30:]

# only start from day 30 for max interval 
# so it can get the past days
draft_df = train_df.copy()

# except the day 
# all the remaining needs to have past values
to_expand_list = draft_df.columns[1:]
expanding_list = []
for word in to_expand_list:
    for i in range(1,31):
        to_add = str(word+"-{}".format(i)) # into format like max_temp-1
        expanding_list.append(to_add)

# create the data with 30days past records
i = 30
add_rows = []
while i < train_df.shape[0]:
    index = i-30
    row = []
    for word in expanding_list:
        feature,day_number = word.split('-')
        row.append(train_df.iloc[i-int(day_number)][feature]) # this is max_temp-1 is 1 day before the target day
    add_rows.append(row)
    i += 1

# create the dataframe which has the max_interval records
# size: 162x510
feature_matrix_X_for_max_interval = pd.DataFrame(data=add_rows, columns=expanding_list)

In [4]:
test_X_for_max_interval = test_features.drop(axis=1, columns=['day'])

In [126]:
# For part-1, we only need the ['max_temp', 'max_dew', 'max_humid', 'dailly_cases']
# And with a preset past_cases_interval and past_weather_interval
past_cases_interval = 15
past_weather_interval_temp = 10
past_weather_interval_dew = 15
past_weather_interval_humid = 30

# kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
# Degree of the polynomial kernel function for poly in kernl
# gamma: scale or auto
# C: Regularization parameter. The strength of the regularization is inversely proportional to C
# coef0 useful in poly and sigmoid

kernel = 'poly'
C = 17000
## Set hyper-parameters for the SVM Model
svm_model = SVR()
svm_model.set_params(**{'kernel': kernel, 'degree': 1, 'C': C,
                        'gamma': 'scale', 'coef0': 7.5, 'tol': 2.5, 'epsilon': 0.04})


            
selected_keywords = ['max_temp', 
                     'avg_temp', 
                     'min_temp', 
                     'max_dew', 
                     'avg_dew', 
                     'min_dew', 
                     'max_humid', 
                     'avg_humid', 
                     'min_humid', 
                     'dailly_cases']
selected_columns = []
selected_columns_daily = []
selected_columns_temp = []
selected_columns_dew = []
selected_columns_humid = []
for word in selected_keywords:
    i = 1
    if (word == 'dailly_cases'):
        while (i <= past_cases_interval):
            selected_columns.append(word+"-{}".format(i))
            selected_columns_daily.append(word+"-{}".format(i))
            i +=1
    elif 'temp' in word:
        while (i <= past_weather_interval_temp):
            selected_columns.append(word+"-{}".format(i))
            selected_columns_temp.append(word+"-{}".format(i))
            i +=1
    elif 'dew' in word:
        while (i <= past_weather_interval_dew):
            selected_columns.append(word+"-{}".format(i))
            selected_columns_dew.append(word+"-{}".format(i))
            i +=1
    elif 'humid' in word:
        while (i <= past_weather_interval_humid):
            selected_columns.append(word+"-{}".format(i))
            selected_columns_humid.append(word+"-{}".format(i))
            i +=1
                     
# get the part1 training data
part_2_feature_X = feature_matrix_X_for_max_interval[selected_columns]

# get the part1 testing data
part_2_test_features_X = test_features[selected_columns]





In [127]:
# Model:
svm_model.fit(part_2_feature_X, feature_matrix_y_for_max_interval["dailly_cases"])
preds = svm_model.predict(part_2_test_features_X)

MeanAbsError = mean_absolute_error(preds, ground_truth)
print("cases:{}, temp:{}, dew:{}, humid:{}, MAE:{}".format(past_cases_interval, past_weather_interval_temp,past_weather_interval_dew,past_weather_interval_humid, MeanAbsError))

# from sklearn import ensemble
# params = {'n_estimators': 2000, 'max_depth': 25, 'min_samples_split': 10,
#           'learning_rate': .01, 'loss': 'huber'} 
# clf = ensemble.GradientBoostingRegressor(**params)
# clf.fit(part_2_feature_X,feature_matrix_y_for_max_interval["dailly_cases"])
# preds = np.expand_dims(clf.predict(part_2_test_features_X), axis = 1)
# MeanAbsError = mean_absolute_error(preds, ground_truth)
# print("kernal:{}, MAE:{}".format(kernel, MeanAbsError))

cases:15, temp:10, dew:15, humid:30, MAE:67.09811826331445


In [119]:
list(part_2_feature_X)

['max_temp-1',
 'max_temp-2',
 'max_temp-3',
 'max_temp-4',
 'max_temp-5',
 'max_temp-6',
 'max_temp-7',
 'max_temp-8',
 'max_temp-9',
 'max_temp-10',
 'min_temp-1',
 'min_temp-2',
 'min_temp-3',
 'min_temp-4',
 'min_temp-5',
 'min_temp-6',
 'min_temp-7',
 'min_temp-8',
 'min_temp-9',
 'min_temp-10',
 'max_dew-1',
 'max_dew-2',
 'max_dew-3',
 'max_dew-4',
 'max_dew-5',
 'max_dew-6',
 'max_dew-7',
 'max_dew-8',
 'max_dew-9',
 'max_dew-10',
 'max_dew-11',
 'max_dew-12',
 'max_dew-13',
 'max_dew-14',
 'max_dew-15',
 'min_dew-1',
 'min_dew-2',
 'min_dew-3',
 'min_dew-4',
 'min_dew-5',
 'min_dew-6',
 'min_dew-7',
 'min_dew-8',
 'min_dew-9',
 'min_dew-10',
 'min_dew-11',
 'min_dew-12',
 'min_dew-13',
 'min_dew-14',
 'min_dew-15',
 'max_humid-1',
 'max_humid-2',
 'max_humid-3',
 'max_humid-4',
 'max_humid-5',
 'max_humid-6',
 'max_humid-7',
 'max_humid-8',
 'max_humid-9',
 'max_humid-10',
 'max_humid-11',
 'max_humid-12',
 'max_humid-13',
 'max_humid-14',
 'max_humid-15',
 'max_humid-16',
 'm

In [83]:
calculate_mean_df = pd.DataFrame(columns=['mean'])
for name in list(part_2_feature_X):
    if 'avg_temp' in name:
        calculate_mean_df[name] = part_2_feature_X[name]

In [84]:
calculate_mean_df

,mean,avg_temp-1,avg_temp-2,avg_temp-3,avg_temp-4,avg_temp-5,avg_temp-6,avg_temp-7,avg_temp-8,avg_temp-9,avg_temp-10,avg_temp_mean
0,NaN,31.05,41.65,40.00,42.10,37.90,22.65,28.50,38.85,41.25,42.70,36.665
1,NaN,27.60,31.05,41.65,40.00,42.10,37.90,22.65,28.50,38.85,41.25,35.155
2,NaN,40.00,27.60,31.05,41.65,40.00,42.10,37.90,22.65,28.50,38.85,35.030
3,NaN,45.70,40.00,27.60,31.05,41.65,40.00,42.10,37.90,22.65,28.50,35.715
4,NaN,49.25,45.70,40.00,27.60,31.05,41.65,40.00,42.10,37.90,22.65,37.790
...,...,...,...,...,...,...,...,...,...,...,...,...
157,NaN,85.55,80.10,75.65,78.90,76.90,84.20,86.50,85.00,80.95,72.20,80.595
158,NaN,88.25,85.55,80.10,75.65,78.90,76.90,84.20,86.50,85.00,80.95,82.200
159,NaN,87.00,88.25,85.55,80.10,75.65,78.90,76.90,84.20,86.50,85.00,82.805
160,NaN,81.85,87.00,88.25,85.55,80.10,75.65,78.90,76.90,84.20,86.50,82.490


In [82]:
calculate_mean_df = pd.DataFrame(columns=['mean'])
for name in list(part_2_feature_X):
    if 'avg_temp' in name:
        calculate_mean_df[name] = part_2_feature_X[name]

calculate_mean_df

,mean,avg_temp-1,avg_temp-2,avg_temp-3,avg_temp-4,avg_temp-5,avg_temp-6,avg_temp-7,avg_temp-8,avg_temp-9,avg_temp-10,avg_temp_mean
0,NaN,31.05,41.65,40.00,42.10,37.90,22.65,28.50,38.85,41.25,42.70,36.665
1,NaN,27.60,31.05,41.65,40.00,42.10,37.90,22.65,28.50,38.85,41.25,35.155
2,NaN,40.00,27.60,31.05,41.65,40.00,42.10,37.90,22.65,28.50,38.85,35.030
3,NaN,45.70,40.00,27.60,31.05,41.65,40.00,42.10,37.90,22.65,28.50,35.715
4,NaN,49.25,45.70,40.00,27.60,31.05,41.65,40.00,42.10,37.90,22.65,37.790
...,...,...,...,...,...,...,...,...,...,...,...,...
157,NaN,85.55,80.10,75.65,78.90,76.90,84.20,86.50,85.00,80.95,72.20,80.595
158,NaN,88.25,85.55,80.10,75.65,78.90,76.90,84.20,86.50,85.00,80.95,82.200
159,NaN,87.00,88.25,85.55,80.10,75.65,78.90,76.90,84.20,86.50,85.00,82.805
160,NaN,81.85,87.00,88.25,85.55,80.10,75.65,78.90,76.90,84.20,86.50,82.490


In [55]:
list(part_2_test_features_X)

['max_temp-1',
 'max_temp-2',
 'max_temp-3',
 'max_temp-4',
 'max_temp-5',
 'max_temp-6',
 'max_temp-7',
 'max_temp-8',
 'max_temp-9',
 'max_temp-10',
 'avg_temp-1',
 'avg_temp-2',
 'avg_temp-3',
 'avg_temp-4',
 'avg_temp-5',
 'avg_temp-6',
 'avg_temp-7',
 'avg_temp-8',
 'avg_temp-9',
 'avg_temp-10',
 'min_temp-1',
 'min_temp-2',
 'min_temp-3',
 'min_temp-4',
 'min_temp-5',
 'min_temp-6',
 'min_temp-7',
 'min_temp-8',
 'min_temp-9',
 'min_temp-10',
 'max_dew-1',
 'max_dew-2',
 'max_dew-3',
 'max_dew-4',
 'max_dew-5',
 'max_dew-6',
 'max_dew-7',
 'max_dew-8',
 'max_dew-9',
 'max_dew-10',
 'max_dew-11',
 'max_dew-12',
 'max_dew-13',
 'max_dew-14',
 'max_dew-15',
 'avg_dew-1',
 'avg_dew-2',
 'avg_dew-3',
 'avg_dew-4',
 'avg_dew-5',
 'avg_dew-6',
 'avg_dew-7',
 'avg_dew-8',
 'avg_dew-9',
 'avg_dew-10',
 'avg_dew-11',
 'avg_dew-12',
 'avg_dew-13',
 'avg_dew-14',
 'avg_dew-15',
 'min_dew-1',
 'min_dew-2',
 'min_dew-3',
 'min_dew-4',
 'min_dew-5',
 'min_dew-6',
 'min_dew-7',
 'min_dew-8',
 'm

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 3, 1)
plt.scatter(train_df['avg_humid'], train_labels_df["dailly_cases"])
plt.title('Revenue vs budget');
plt.xlabel('budget')
plt.ylabel('revenue')

# plt.subplot(1, 3, 2)
# plt.scatter(np.log(train_df['avg_temp']), np.log(train_labels_df["dailly_cases"]))
# plt.xlabel('log_budget')
# plt.ylabel('log_revenue')
# plt.title('Log Revenue vs Log budget');

# plt.subplot(1, 3, 3)
# plt.scatter((train_df['avg_temp'])**(1/2), (train_labels_df["dailly_cases"])**(1/2))
# plt.xlabel('log_budget')
# plt.ylabel('log_revenue')
# plt.title('Square(Log Revenue) vs Square(log budget)');

In [ ]:
max(train_labels_df["dailly_cases"])